In [1]:
import pandas as pd
import numpy as np
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import re
import nltk 
import string

In [2]:
data = pd.read_csv("data2.csv")
data.sample(3)

,Data
56,"TRIBUNNEWS.COM, JAKARTA -Imam Besar Front Pemb..."
15,Peningkatan jumlah kasus positif Covid-19 di J...
11,Pemerintah KotaYogyakartamengumpulkan tujuh di...


In [3]:
sendec = [0 for x in range(100)]
arr = 0
for i in data["Data"]:
    sendec[arr] = i.split(".")
    arr = arr+1
sendec[1]

['Juru Bicara Satuan Tugas Penanganan Covid-19 Wiku Adisasmito mengingatkan masyarakat tak mendiskriminasi pasien Covid-19 yang ada di lingkungan mereka',
 'Menurut Wiku, terinfeksi virus corona bukanlah sebuah aib',
 ' Dukungan kepada pasien justru sangat dibutuhkan untuk membantu proses penyembuhan',
 '"Jangan ada sikap diskriminatif kepada keluarga atau kerabat yang didapati positif Covid-19, positif Covid bukanlah aib," kata Wiku dalam konferensi pers yang ditayangkan YouTube Sekretariat Presiden, Selasa (12AQzAZ1/12/2020)',
 '"Sebaliknya, dukungan kepada mereka yang menderita Covid-19 dapat membantu proses penyembuhan mereka," tuturnya',
 'Baca juga:Sepanjang Rumah Sakit Declare, Kemenkes: Biaya Perawatan Pasien Covid-19 Ditanggung NegaraDalam kesempatan yang sama, Wiku juga meminta masyarakat mendukung upayatracingyang tengah digenjot pemerintah',
 'Tracingmerupakan penelusuran kontak terdekat pasien Covid-19',
 'Upayatracing, kata Wiku, sama pentingnya dengantestingdantreatment'

In [4]:
sendecprepos = [0 for x in range(100)]
for i in range(len(sendec)):
    gabung = []
    for j in sendec[i]:
        #print(j)
        #Huruf Kecil + Tokenize
        lowertext = j.lower()
        j = re.sub("[^a-zA-Z0-9_]", " ",str(lowertext))
        words = j.split()
        
        #Stopwords removal
        stop_words = [line.rstrip('\n') for line in open('stopwordlist.txt')]
        filtered_text = [word for word in words if word not in stop_words]
        
        #StemmingSastrawi
        factory = StemmerFactory()
        stemmer = factory.create_stemmer()
        stemming = []
        for a in filtered_text:
            b   = stemmer.stem(a)
            stemming.append(b)
        a = " ".join(w for w in stemming)
        gabung.append(a)
    sendecprepos[i] = gabung
sendecprepos[0]

['ketua bidang data teknologi informasi satu tugas penanganancovid 19dewi nur aisyahmengatakan angkapositivity ratecovid 19 indonesia alami turun oktober november 2020',
 'data satgas positivity ratecovid 19 indonesia capai 16 69 persen september',
 'turun angka 14 26 persen oktober november turun 13 55 persen dewi konferensi pers daring rabu 2 12 2020',
 'baca rizieq shihab maaf janji kerumun pandemi covid 19positivity rateadalah banding positif covid 19 sampel periksa metode swab test',
 'kendati alami turun tingkatpositivity ratecovid 19 tanah air bilang banding standar organisasi sehat dunia who',
 'rekomendasi who pandemi covid 19 kendali jikapositivity ratemaksimal 5 persen',
 'tren turun oktober november standar who',
 'standar who kendali target 5 persen orang periksa dewi',
 'dewi periksa orang indonesia capai target tetap who',
 'baca luhut insya allah pasok vaksin covid 19 desembertarget periksa who 1 juta duduk periksa 1',
 '000 orang pekan',
 'asumsi duduk 267 juta jiwa in

In [5]:
np.save("sendecprepos", sendecprepos)

In [4]:
sendecprepos = np.load('sendecprepos.npy', allow_pickle=True)

In [5]:
noun = pd.read_csv("noun.csv")
noun

,lema,nilai,nilai1
0,A,huruf pertama abjad Indonesia,nama huruf a
1,ab,wadah kecil dari timah untuk candu,hap
2,abad,masa seratus tahun,jangka waktu yang lamanya seratus tahun
3,abah,arah,tuju
4,abah-abah,alat,perkakas
...,...,...,...
9000,zirkonium,(Kim) logam tanah langka berwarna putih perak ...,unsur dengan nomor atom 40 lambang Zr dan bobo...
9001,zona,salah satu dari lima bagian besar permukaan bu...,jalur iklim
9002,zuhur,waktu tengah hari,waktu salat wajib setelah matahari tergelincir...
9003,zuriah,(Ar) keturunan,anak cucu


In [6]:
nounfilter = [0 for x in range(100)]
for i in range(len(sendecprepos)):
    gabung = []
    for j in sendecprepos[i]:
        words = j.split()
        ceknoun = [line.rstrip('\n') for line in noun["lema"]]
        filtered_text = [word for word in words if word in ceknoun]
        a = " ".join(w for w in filtered_text)
        gabung.append(a)
    nounfilter[i] = gabung
nounfilter[0]

['ketua bidang data teknologi informasi tugas',
 'data',
 'angka dewi pers',
 'maaf janji banding sampel metode',
 'tanah air banding standar organisasi dunia',
 'rekomendasi',
 'standar',
 'standar orang dewi',
 'dewi orang',
 'pasok',
 'orang pekan',
 'asumsi jiwa',
 'orang',
 'dewi orang',
 'pekan angka',
 'jalan',
 '']

In [7]:
kal = [0 for x in range(len(nounfilter))]#kalimat berita keseluruhan
kalnodup = [0 for x in range(len(nounfilter))]#kalimat berita non duplicate

for i in range(len(nounfilter)):
    kal[i] = " ".join(nounfilter[i])
    l = kal[i].split()
    l.sort()
    k = []
    
    for j in l:
        #print(j)
        if (kal[i].count(j)>1 and (j not in k)or kal[i].count(j)==1): 
            k.append(j)
        kalnodup[i] = k
kalnodup[0]

['air',
 'angka',
 'asumsi',
 'banding',
 'bidang',
 'data',
 'dewi',
 'dunia',
 'informasi',
 'jalan',
 'janji',
 'jiwa',
 'ketua',
 'maaf',
 'metode',
 'orang',
 'organisasi',
 'pasok',
 'pekan',
 'pers',
 'rekomendasi',
 'sampel',
 'standar',
 'tanah',
 'teknologi',
 'tugas']

In [8]:
ambigu = [0 for x in range(len(kalnodup))]
#ambigu = [[0 for x in range(len(kalnodup))]for xx in range(2)]
#cobas = [0 for x in range(len(kalnodup))]

for i in range(len(kalnodup)):
    k = [] #array kalimat ambigu
    l = [] #array total kalimat ambigu
    
    for j in kalnodup[i]:

        m = kal[i].count(j)
            
        if (m>1 and j!="as"):
            k.append(j)
            l.append(m)
       
    tt = max(l) #nilai ambigu tertinggi
    uu = l.index(tt) #index nilai ambigu tertinggi
    #ambigu[i][0] = k[uu]
    #ambigu[i][1] = l[uu]
    ambigu[i] = k[uu]
print(ambigu[0])

orang


In [9]:
pop = []
pop1 = []
ambdex = [] #index kata ambigu
for i in range(len(nounfilter)):
    totalnilai = 0
    totalnilai1 = 0
    for j in range(len(nounfilter[i])):
        if (nounfilter[i][j].count(ambigu[i])==1):
            #print(nounfilter[i][j])
            #totalnilai = 0
            #totalnilai1 = 0
            for k in set(nounfilter[i][j].split()):
                if k!=ambigu[i]:
                    cek1 = ceknoun.index(ambigu[i])
                    cek2 = ceknoun.index(k)
                    #print(cek2)
                    #print(k)
                    
                    for l in set(noun["nilai"][cek1].split()):
                        totalnilai = totalnilai + noun["nilai"][cek2].count(l)
                    for m in set(noun["nilai1"][cek1].split()):
                        totalnilai1 = totalnilai1 + noun["nilai1"][cek2].count(m)
    pop.append(totalnilai)
    pop1.append(totalnilai1)
    ambdex.append(cek1)
            #pop.append(totalnilai)
            #pop1.append(totalnilai1)
print(pop)
print(pop1)
print(ambdex)
print(noun["lema"][4984])

[0, 14, 15, 0, 0, 11, 13, 7, 0, 6, 28, 33, 0, 7, 1, 32, 41, 1, 1, 2, 20, 0, 7, 3, 0, 0, 44, 13, 12, 22, 24, 12, 0, 0, 0, 37, 11, 14, 2, 12, 4, 1, 30, 6, 12, 26, 0, 9, 34, 0, 37, 17, 30, 26, 0, 0, 33, 17, 11, 5, 0, 1, 5, 28, 2, 0, 8, 54, 16, 0, 5, 0, 6, 21, 38, 0, 23, 55, 5, 1, 10, 10, 0, 1, 1, 3, 6, 0, 8, 20, 18, 0, 0, 0, 0, 1, 0, 4, 20, 0]
[3, 2, 0, 0, 3, 0, 0, 0, 7, 0, 35, 6, 5, 7, 4, 13, 0, 1, 2, 0, 0, 0, 1, 6, 0, 0, 22, 5, 5, 35, 5, 2, 6, 0, 0, 12, 5, 5, 0, 0, 1, 0, 0, 0, 3, 4, 0, 11, 9, 0, 16, 8, 8, 0, 3, 63, 0, 6, 3, 0, 0, 0, 4, 0, 4, 91, 0, 0, 0, 7, 0, 0, 0, 0, 30, 0, 18, 25, 7, 0, 7, 0, 0, 0, 1, 2, 0, 0, 0, 10, 13, 0, 27, 0, 0, 1, 7, 0, 0, 0]
[5116, 8622, 2620, 1424, 5116, 8154, 7198, 2620, 5116, 8578, 8886, 7177, 5116, 6797, 5116, 1424, 6395, 3355, 1835, 306, 2348, 1954, 7959, 3580, 3580, 3580, 2092, 2092, 2092, 4984, 8448, 8725, 1622, 7177, 7177, 1603, 4984, 2092, 3999, 2620, 5200, 4728, 36, 4728, 4024, 2092, 2092, 4024, 4654, 2275, 2657, 8448, 7443, 8684, 5116, 1663, 36, 402

In [10]:
senex = [0 for x in range(100)]
for i in range(len(sendec)):
    temp = []
    for j in range(len(sendec[i])):
        lgth = 0 #length
        dst = 0 #nilai destinct member
        cek = 0
        #lexi = 0
        for k in set(sendec[i][j].split()):
            #print(k)
            lgth = lgth + 1
            if pop>pop1:
                cek = cek + noun["nilai"][ambdex[i]].count(k)
            else:
                cek = cek + noun["nilai1"][ambdex[i]].count(k)
        dst = lgth - cek
        lexi = (1-(dst/lgth))*lgth if lgth != 0 else 0
        senextemp = 1+(lexi/len(sendec[i])) #sentence extrator
        temp.append(senextemp)
    senex[i] = temp
    
print(lgth)
print(dst)
print(cek)
print(senex[1])

6
6
0
[1.1111111111111112, 1.0, 1.0, 1.0555555555555556, 1.0555555555555556, 1.0555555555555556, 1.0, 1.0, 1.0, 1.1111111111111112, 1.0, 1.1111111111111112, 1.0, 1.0, 1.0555555555555556, 1.1111111111111112, 1.0, 1.0]


In [11]:
hasil = []
for i in range(len(sendec)):
    for j in range(len(sendec[i])):
        a = sendec[i]
        b = senex[i]
        zipped_lists = zip(a, b)
        sorted_pairs = sorted(zipped_lists,reverse=True)

        tuples = zip(*sorted_pairs)
        a, b = [ list(tuple) for tuple in  tuples]
    hasil.append(a)
hasil[0]

['Selain itu, Dewi mengatakan saat ini jumlah pemeriksaan orang di Indonesia sudah hampir mencapai target yang ditetapkan WHO',
 'Per November ini, Dewi menyebut jumlah pemeriksaan orang di Indonesia sudah 90 persen dari target WHO',
 'Menurut rekomendasi WHO, pandemi Covid-19 bisa dikatakan terkendali jikapositivity ratemaksimal 5 persen',
 'Menurut data Satgas,positivity rateCovid-19 di Indonesia sempat mencapai 16,69 persen pada September',
 'Ketua Bidang Data dan Teknologi Informasi Satuan Tugas PenangananCovid-19Dewi Nur Aisyahmengatakan angkapositivity rateCovid-19 di Indonesia mengalami penurunan sejak Oktober hingga November 2020',
 'Kendati mengalami penurunan, dia mengatakan tingkatpositivity rateCovid-19 di Tanah Air masih terbilang tinggi jika dibandingkan dengan standar Organisasi Kesehatan Dunia (WHO)',
 'Baca juga:Rizieq Shihab Minta Maaf, Janji Tidak Buat Kerumunan Lagi Selama Pandemi Covid-19Positivity rateadalah perbandingan antara kasus positif Covid-19 dan jumlah sa

In [12]:
rangkuman1 = []
for i in range(len(hasil)):
    persen = round(len(hasil[i])*(30/100))
    #print((persen))
    temp = []
    for j in range(persen):
        temp.append(hasil[i][j])
    
    x = " ".join(w for w in temp)
    rangkuman1.append(x)
rangkuman1[0]

'Selain itu, Dewi mengatakan saat ini jumlah pemeriksaan orang di Indonesia sudah hampir mencapai target yang ditetapkan WHO Per November ini, Dewi menyebut jumlah pemeriksaan orang di Indonesia sudah 90 persen dari target WHO Menurut rekomendasi WHO, pandemi Covid-19 bisa dikatakan terkendali jikapositivity ratemaksimal 5 persen Menurut data Satgas,positivity rateCovid-19 di Indonesia sempat mencapai 16,69 persen pada September Ketua Bidang Data dan Teknologi Informasi Satuan Tugas PenangananCovid-19Dewi Nur Aisyahmengatakan angkapositivity rateCovid-19 di Indonesia mengalami penurunan sejak Oktober hingga November 2020'

In [13]:
rangkuman2 = []
for i in range(len(hasil)):
    persen = round(len(hasil[i])*(25/100))
    #print((persen))
    temp = []
    for j in range(persen):
        temp.append(hasil[i][j])
    
    x = " ".join(w for w in temp)
    rangkuman2.append(x)
rangkuman2[0]

'Selain itu, Dewi mengatakan saat ini jumlah pemeriksaan orang di Indonesia sudah hampir mencapai target yang ditetapkan WHO Per November ini, Dewi menyebut jumlah pemeriksaan orang di Indonesia sudah 90 persen dari target WHO Menurut rekomendasi WHO, pandemi Covid-19 bisa dikatakan terkendali jikapositivity ratemaksimal 5 persen Menurut data Satgas,positivity rateCovid-19 di Indonesia sempat mencapai 16,69 persen pada September'

In [14]:
rangkuman3 = []
for i in range(len(hasil)):
    persen = round(len(hasil[i])*(20/100))
    #print((persen))
    temp = []
    for j in range(persen):
        temp.append(hasil[i][j])
    
    x = " ".join(w for w in temp)
    rangkuman3.append(x)
rangkuman3[0]

'Selain itu, Dewi mengatakan saat ini jumlah pemeriksaan orang di Indonesia sudah hampir mencapai target yang ditetapkan WHO Per November ini, Dewi menyebut jumlah pemeriksaan orang di Indonesia sudah 90 persen dari target WHO Menurut rekomendasi WHO, pandemi Covid-19 bisa dikatakan terkendali jikapositivity ratemaksimal 5 persen'

In [23]:
rangkuman4 = []
for i in range(len(hasil)):
    persen = round(len(hasil[i]))
    #print((persen))
    temp = []
    for j in range(persen):
        temp.append(hasil[i][j])
    
    x = " ".join(w for w in temp)
    rangkuman4.append(x)
rangkuman4[0]

'Selain itu, Dewi mengatakan saat ini jumlah pemeriksaan orang di Indonesia sudah hampir mencapai target yang ditetapkan WHO Per November ini, Dewi menyebut jumlah pemeriksaan orang di Indonesia sudah 90 persen dari target WHO Menurut rekomendasi WHO, pandemi Covid-19 bisa dikatakan terkendali jikapositivity ratemaksimal 5 persen Menurut data Satgas,positivity rateCovid-19 di Indonesia sempat mencapai 16,69 persen pada September Ketua Bidang Data dan Teknologi Informasi Satuan Tugas PenangananCovid-19Dewi Nur Aisyahmengatakan angkapositivity rateCovid-19 di Indonesia mengalami penurunan sejak Oktober hingga November 2020 Kendati mengalami penurunan, dia mengatakan tingkatpositivity rateCovid-19 di Tanah Air masih terbilang tinggi jika dibandingkan dengan standar Organisasi Kesehatan Dunia (WHO) Baca juga:Rizieq Shihab Minta Maaf, Janji Tidak Buat Kerumunan Lagi Selama Pandemi Covid-19Positivity rateadalah perbandingan antara kasus positif Covid-19 dan jumlah sampel yang diperiksa denga

In [15]:
sendec[1]

['Juru Bicara Satuan Tugas Penanganan Covid-19 Wiku Adisasmito mengingatkan masyarakat tak mendiskriminasi pasien Covid-19 yang ada di lingkungan mereka',
 'Menurut Wiku, terinfeksi virus corona bukanlah sebuah aib',
 ' Dukungan kepada pasien justru sangat dibutuhkan untuk membantu proses penyembuhan',
 '"Jangan ada sikap diskriminatif kepada keluarga atau kerabat yang didapati positif Covid-19, positif Covid bukanlah aib," kata Wiku dalam konferensi pers yang ditayangkan YouTube Sekretariat Presiden, Selasa (12AQzAZ1/12/2020)',
 '"Sebaliknya, dukungan kepada mereka yang menderita Covid-19 dapat membantu proses penyembuhan mereka," tuturnya',
 'Baca juga:Sepanjang Rumah Sakit Declare, Kemenkes: Biaya Perawatan Pasien Covid-19 Ditanggung NegaraDalam kesempatan yang sama, Wiku juga meminta masyarakat mendukung upayatracingyang tengah digenjot pemerintah',
 'Tracingmerupakan penelusuran kontak terdekat pasien Covid-19',
 'Upayatracing, kata Wiku, sama pentingnya dengantestingdantreatment'

In [16]:
from rouge_score import rouge_scorer

In [17]:
rougeL1=[]
for i in range(len(rangkuman1)):
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    scores = scorer.score(data["Data"][i], rangkuman1[i])
    rougeL1.append(scores["rougeL"])
hasilrouge1 = pd.DataFrame(rougeL1)
print(hasilrouge1)
print(hasilrouge1.mean())

    precision    recall  fmeasure
0    0.406977  0.132576  0.200000
1    0.666667  0.118056  0.200590
2    0.735849  0.289963  0.416000
3    0.581395  0.234742  0.334448
4    0.645161  0.221729  0.330033
..        ...       ...       ...
95   0.726027  0.218107  0.335443
96   0.569231  0.256944  0.354067
97   0.650000  0.178082  0.279570
98   0.746479  0.256039  0.381295
99   0.860000  0.191964  0.313869

[100 rows x 3 columns]
precision    0.614510
recall       0.192948
fmeasure     0.291641
dtype: float64


In [18]:
rougeL2=[]
for i in range(len(rangkuman2)):
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    scores = scorer.score(data["Data"][i], rangkuman2[i])
    rougeL2.append(scores["rougeL"])
hasilrouge2 = pd.DataFrame(rougeL2)
print(hasilrouge2)
print(hasilrouge2.mean())

    precision    recall  fmeasure
0    0.564516  0.132576  0.214724
1    0.790698  0.118056  0.205438
2    0.658537  0.200743  0.307692
3    0.689655  0.187793  0.295203
4    0.793103  0.203991  0.324515
..        ...       ...       ...
95   0.812500  0.106996  0.189091
96   0.616667  0.256944  0.362745
97   0.650000  0.178082  0.279570
98   0.704918  0.207729  0.320896
99   1.000000  0.191964  0.322097

[100 rows x 3 columns]
precision    0.660944
recall       0.169268
fmeasure     0.266947
dtype: float64


In [19]:
rougeL3=[]
for i in range(len(rangkuman1)):
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    scores = scorer.score(data["Data"][i], rangkuman3[i])
    rougeL3.append(scores["rougeL"])
hasilrouge3 = pd.DataFrame(rougeL3)
print(hasilrouge3)
print(hasilrouge3.mean())

    precision    recall  fmeasure
0    0.744681  0.132576  0.225080
1    0.790698  0.118056  0.205438
2    0.550000  0.122677  0.200608
3    0.860465  0.173709  0.289062
4    0.760000  0.168514  0.275862
..        ...       ...       ...
95   0.812500  0.106996  0.189091
96   0.621622  0.239583  0.345865
97   0.650000  0.178082  0.279570
98   1.000000  0.178744  0.303279
99   1.000000  0.093750  0.171429

[100 rows x 3 columns]
precision    0.701970
recall       0.143046
fmeasure     0.234784
dtype: float64


In [24]:
rougeL4=[]
for i in range(len(rangkuman1)):
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    scores = scorer.score(data["Data"][i], rangkuman4[i])
    rougeL4.append(scores["rougeL"])
hasilrouge4 = pd.DataFrame(rougeL4)
print(hasilrouge4)
print(hasilrouge4.mean())

    precision    recall  fmeasure
0    0.446970  0.446970  0.446970
1    0.475694  0.475694  0.475694
2    0.397770  0.397770  0.397770
3    0.530516  0.530516  0.530516
4    0.339246  0.339246  0.339246
..        ...       ...       ...
95   0.493827  0.493827  0.493827
96   0.479167  0.479167  0.479167
97   0.547945  0.547945  0.547945
98   0.429952  0.429952  0.429952
99   0.531250  0.531250  0.531250

[100 rows x 3 columns]
precision    0.409765
recall       0.409765
fmeasure     0.409765
dtype: float64


In [26]:
arr1 = hasilrouge1.mean()
arr2 = hasilrouge2.mean()
arr3 = hasilrouge3.mean()
arr4 = hasilrouge4.mean()

In [27]:
allarr = pd.DataFrame({'30%':arr1, '25%':arr2, '20%':arr3, '0':arr4,})

In [28]:
allarr

,30%,25%,20%,0
precision,0.614510,0.660944,0.701970,0.409765
recall,0.192948,0.169268,0.143046,0.409765
fmeasure,0.291641,0.266947,0.234784,0.409765
